# Download OSM Data from Web #

This method uses de OSM Overpass API to get OSM data from the Web

In [ ]:
from gasp3.gt.to.osm import download_by_boundary

inBoundary = '/home/jasp/flainar/lmt_centro.shp'
outFolder  = '/home/jasp/flainar'
outName    = 'osm_centro'

EPSG = 3763

download_by_boundary(
    inBoundary, outFolder, outName, EPSG,
    GetUrl=True, conPSQL=None, geomCol=None,
    justOneFeature=None
)

# Convert OSM data into Feature Class #

In [ ]:
from gasp3.gt.to.shp import osm_to_featcls

osm_to_featcls(
    '/home/jasp/flainar/portugal-latest.osm.pbf',
    '/home/jasp/flainar/', useXmlName=True, outepsg=32629)

### OSM File to PSQL DB ###

In [ ]:
from gasp3.sql.to import osm_to_pgsql

conDB = {
    'HOST' : 'localhost', 'PORT' : '5433', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'DATABASE' : 'flainar_db'
}

inOsm = '/home/jasp/flainar/osm_centro.xml'

osm_to_pgsql(inOsm, conDB)

# OSM File to Relation Sctruture: #

In [ ]:
from gasp3.sql.to.osm import osm_to_relationaldb

conDB = {
    'HOST' : 'localhost', 'PORT' : '5433', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'DATABASE' : 'flainar_osm',
    'TEMPLATE' : 'postgis_template'
}

inOsm = '/home/jasp/flainar/osm_centro.xml'

inSchema = {
    "TBL" : 'points', 'FID' : 'ogc_fid',
    "COLS" : [
        'name', 'osm_id',
        "ST_X(wkb_geometry) AS longitude",
        "ST_Y(wkb_geometry) AS latitude",
        "wkb_geometry AS geom",
        "NULL AS featurecategoryid",
        "NULL AS flainarcategoryid",
        "NULL AS createdby",
        "NOW() AS createdon",
        "NULL AS updatedon",
        "NULL AS deletedon"
    ],
    "NOT_KEYS" : [
        'ogc_fid', 'osm_id', 'name', "wkb_geometry",
        'healthcare2', 'other_tags'
    ]
}

osmGeoTbl = {"TBL" : 'position', "FID" : 'positionid'}

osmCatTbl = {
    "TBL" : 'osmcategory', "FID" : "osmcategoryid",
    "KEY_COL" : "keycategory", "VAL_COL" : "value",
    "COLS" : [
        "NULL AS createdby", "NOW() AS createdon",
        "NULL AS updatedon", "NULL AS deletedon"
    ]
}

osmRelTbl = {
    "TBL" : "position_osmcat", "FID" : 'pososmcatid'
}

outsql = '/home/jasp/flainar/flainar_osm.sql'
osm_to_relationaldb(conDB, inOsm, inSchema, osmGeoTbl, osmCatTbl, osmRelTbl, outSQL=outsql)

In [ ]:
"""
Clip OSM File
"""

from gasp3.gt.prop.ext import get_ext

from gasp3 import exec_cmd

inBoundary = '/home/jasp/flainar/lmt_centro.shp'
inOsm = '/home/jasp/flainar/portugal-latest.osm.pbf'
ouOsm = '/home/jasp/flainar/osm_centro.xml'

EPSG = 3763

extent = get_ext(inBoundary, outEpsg=4326)


rcmd = exec_cmd((
    "osmosis --read-pbf {} "
    "--bounding-box top={} left={} bottom={} right={} "
    "--write-xml {}"
).format(
    inOsm, str(extent[3]), str(extent[0]), str(extent[2]),
    str(extent[1]), ouOsm
))